In [2]:
import sys
import os
import json
import pickle
from mmdet3d.datasets.pipelines import Compose
import mmcv
import numpy as np

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# change working directory
os.chdir(
    "/home/ensai/Documents/msd06-1-smart-data-project/LIDC/MV2Dmain/MV2Dbasedlifting"
)
print("Current Working Directory:", os.getcwd())

from mmdet3d_plugin import CustomLIDCDataset

data_root_lidc = "./data/lidc/"

lidc_dataclass = CustomLIDCDataset(
    data_root=data_root_lidc,
    ann_file=data_root_lidc + "toy_infos.pkl",
    ann_file_2d=data_root_lidc + "toy_infos_2d_anno.coco.json",
)

Current Working Directory: /home/ensai/Documents/msd06-1-smart-data-project/LIDC/MV2Dmain/MV2Dbasedlifting
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [5]:
test_pipeline = [
    dict(type="LoadMultiViewImageFromFiles", to_float32=True),
    dict(
        type="LoadAnnotationsMono3D",
        with_bbox_3d=False,
        with_label_3d=False,
        with_bbox_2d=False,
        with_attr_label=False,
    ),
    dict(type="PadMultiViewImage", size_divisor=32),
    dict(
        type="DefaultFormatBundleMono3D", class_names=["nodule"], with_label=False
    ),  # Follow train_pipeline, without label
    dict(type="CollectMono3DLIDC", debug=False, keys=["img"]),
]

pipeline = Compose(test_pipeline)

In [3]:
#
# Read outputs from file
#
with open("eval_result/outputs.pkl", "rb") as f:
    outputs = pickle.load(f)

len(outputs)

# for i, result in enumerate(outputs):
#     point = lidc_dataclass._extract_data(i, pipeline=pipeline, key="point")
#     print(point)

2

In [28]:
outputs[0]["pts_bbox"]

{'boxes_3d': CameraInstance3DBoxes(
     tensor([[ 5.1162e+01, -5.1084e+01,  8.6403e-01,  7.5836e+00,  7.8858e+00,
           4.2698e+00,  7.9395e-04],
         [ 5.1182e+01, -5.0714e+01, -7.5548e+00,  8.5807e+00,  9.4250e+00,
           5.1095e+00,  4.0444e-03],
         [-5.1200e+01, -5.0538e+01, -7.4969e+00,  8.4107e+00,  9.2014e+00,
           4.9939e+00,  5.9246e-03],
         [ 5.1186e+01, -5.0212e+01, -7.4752e+00,  8.3234e+00,  9.0175e+00,
           4.9504e+00,  5.3716e-03],
         [-5.1200e+01, -4.7626e+01,  3.3550e-01,  8.5219e+00,  9.1917e+00,
           5.3254e+00, -4.7433e-03],
         [ 5.1195e+01, -4.3937e+01,  2.6966e-01,  8.6033e+00,  9.1274e+00,
           5.4566e+00, -5.1695e-03],
         [ 5.1195e+01, -4.3906e+01,  2.6909e-01,  8.6038e+00,  9.1272e+00,
           5.4578e+00, -5.1612e-03],
         [-5.1200e+01, -4.3841e+01,  2.6393e-01,  8.6031e+00,  9.1283e+00,
           5.4681e+00, -5.0911e-03],
         [ 5.1195e+01, -4.3455e+01, -7.7314e+00,  8.5978e+00,  9

In [9]:
result_files, tmp_dir = lidc_dataclass.format_results(outputs)


Formating bboxes of pts_bbox
Start to convert detection format...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2/2, 32.1 task/s, elapsed: 0s, ETA:     0s
Results writes to /tmp/tmp6cub750q/results/pts_bbox/results_lidc.json


In [ ]:
pts_box = result_files["pts_bbox"]

with open(result_files["pts_bbox"], "rb") as f:
    result_lidc = json.load(f)

len(result_lidc["results"]["0"])

result_0 = result_lidc["results"]["0"]

result_lidc["results"]["0"]

[{'sample_token': 0,
  'translation': [51.1621208190918, -55.027366638183594, 0.8640260696411133],
  'size': [7.583618640899658, 7.8857927322387695, 4.269811630249023],
  'rotation': [0.999999921204919, 0.0, 0.0, 0.0003969762661070855],
  'velocity': [0, 0],
  'detection_name': 'nodule',
  'detection_score': 0.23275133967399597,
  'attribute_name': None},
 {'sample_token': 0,
  'translation': [51.18184280395508, -55.425994873046875, -7.554758071899414],
  'size': [8.580738067626953, 9.424960136413574, 5.109525680541992],
  'rotation': [0.9999979553273762, 0.0, 0.0, 0.0020222119242977858],
  'velocity': [0, 0],
  'detection_name': 'nodule',
  'detection_score': 0.195853129029274,
  'attribute_name': None},
 {'sample_token': 0,
  'translation': [-51.19993591308594, -55.13853454589844, -7.49692440032959],
  'size': [8.410650253295898, 9.201355934143066, 4.993858337402344],
  'rotation': [0.999995612339194, 0.0, 0.0, 0.0029623136836660065],
  'velocity': [0, 0],
  'detection_name': 'nodule

In [24]:
# gt_bbox_3d = lidc_dataclass.get_ann_info(0)["gt_bboxes_3d"]
# gt_bbox_3d

img_path = lidc_dataclass.get_data_info(0)["img_filename"][0]
print(img_path)
img = mmcv.imread(img_path)
img = np.array(img)
img.shape

intrinsics = lidc_dataclass.get_data_info(0)["intrinsics"]
len(intrinsics)
cam00_intrinsic = intrinsics[0]

./data/lidc/Images/Patient0000/Image_00.png


In [30]:
gt_bbox_3d = lidc_dataclass.get_data_info(0)["ann_info"]["gt_bboxes_3d"]
gt_bbox_3d

CameraInstance3DBoxes(
    tensor([[77.4891, 58.1531, 56.3000, 34.4531, 28.1250, 17.5000,  0.0000],
        [77.4891, 60.2625, 57.5500, 30.2344, 29.5312, 15.0000,  0.0000],
        [78.1922, 57.4500, 56.3000, 31.6406, 26.7188, 17.5000,  0.0000],
        [78.1922, 57.4500, 57.5500, 35.8594, 26.7188, 20.0000,  0.0000]]))

In [ ]:
gt_bbox_2d = lidc_dataclass.get_data_info(0)["ann_info"]["gt_bboxes_2d"][0]
len(gt_bbox_2d)
gt_bbox_2d = gt_bbox_2d

test_bbox = np.array([[567, 369, 567 + 70, 369 + 54]])


mmcv.imshow_bboxes(img, test_bbox, out_file="p0000cam00_wbbox.png")

In [190]:
from mmdet3d.core.bbox import (
    Box3DMode,
    Coord3DMode,
    LiDARInstance3DBoxes,
    CameraInstance3DBoxes,
)
from mmdet3d.core.bbox import points_cam2img

corners_3d = gt_bbox_3d.corners

num_bbox = corners_3d.shape[0]

points_3d = corners_3d.reshape(-1, 3)
points_3d.shape


cam2img = cam00_intrinsic

# project to 2d to get image coords (uv)
uv_origin = points_cam2img(points_3d, cam2img)

uv_origin = (uv_origin - 1).round()
imgfov_pts_2d = uv_origin[..., :2].reshape(num_bbox, 8, 2).numpy()

imgfov_pts_2d[0, :, :]

array([[ 664.,  197.],
       [ 333.,   11.],
       [ 333.,  310.],
       [ 664.,  631.],
       [1196.,  197.],
       [ 700.,   11.],
       [ 700.,  310.],
       [1196.,  631.]], dtype=float32)

In [29]:
from mmdet3d.core.visualizer.open3d_vis import Visualizer
from mmdet3d.core.visualizer.image_vis import draw_camera_bbox3d_on_img
from mmdet3d.core.visualizer import show_multi_modality_result

show_multi_modality_result(
    img,
    gt_bbox_3d,
    gt_bbox_3d,
    cam00_intrinsic,
    box_mode="camera",
    out_dir="eval_result",
    filename="test",
)

# test = draw_camera_bbox3d_on_img(gt_bbox_3d, img, cam00_intrinsic, img_metas=None)
# mmcv.imshow(test)